# need to fill in any routes with BOTH missing (known) length AND route (so that we can have a guesstimate for all)

# Import packages and data

In [1]:
# note that the order of points in GeoPandas is longitude, latitude 
# (opposite order from that of many data sets)

import geopandas
import shapely.geometry
import shapely.ops
import pyproj
import pandas

import time
import numpy

import pygsheets

import EEZ file

In [2]:
# from https://www.marineregions.org/downloads.php
# in the section "Marine and land zones: the union of world country boundaries and EEZ's"
eez_file = '/Users/baird/Dropbox/_gis-data/eez/EEZ_land_union_v2_201410/EEZ_land_v2_201410.shp'

eez_and_land_boundaries_gdf = geopandas.read_file(eez_file)
eez_and_land_boundaries_gdf = eez_and_land_boundaries_gdf.set_index('Country')
#eez_4087 = eez_and_land_boundaries.to_crs('epsg:4087')

### special cases for EEZs (Hong Kong, Macao...)

import natural earth data file to pick out hong kong, macao

In [3]:
nat_earth_file = '/Users/baird/Dropbox/_gis-data/_natural_earth_data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'
nat_earth_gdf = geopandas.read_file(nat_earth_file)

# pull out shapely geometry polygons and multipolygons
china_geom = eez_and_land_boundaries_gdf.loc[eez_and_land_boundaries_gdf.index=='China']['geometry'].values[0]
hk_geom = nat_earth_gdf.loc[nat_earth_gdf.ADMIN=='Hong Kong S.A.R.']['geometry'].values[0]
macao_geom = nat_earth_gdf.loc[nat_earth_gdf.ADMIN=='Macao S.A.R']['geometry'].values[0]

china_new_geom = china_geom - china_geom.intersection(hk_geom)
china_new_geom = china_new_geom - china_new_geom.intersection(macao_geom)

check that the new geometry is smaller in area

now replace the original china in EEZ file, and add Hong Kong, Macao

In [4]:
empty_row_hk = geopandas.GeoDataFrame([[numpy.nan]*eez_and_land_boundaries_gdf.columns.size],
                             columns=eez_and_land_boundaries_gdf.columns, index=['Hong Kong'])
empty_row_hk['geometry'] = hk_geom

empty_row_macao = geopandas.GeoDataFrame([[numpy.nan]*eez_and_land_boundaries_gdf.columns.size],
                             columns=eez_and_land_boundaries_gdf.columns, index=['Macao'])
empty_row_macao['geometry'] = macao_geom

# add geometries to these rows
eez_and_land_boundaries_gdf = eez_and_land_boundaries_gdf.append(empty_row_hk)
eez_and_land_boundaries_gdf = eez_and_land_boundaries_gdf.append(empty_row_macao)
# replace with new version of China
eez_and_land_boundaries_gdf.loc[eez_and_land_boundaries_gdf.index=='China','geometry'] = china_new_geom

now create a blob for all boundaries

In [5]:
# create one blob for all world land and EEZ boundaries, using Shapely function cascaded_union 
# whatever is left out is, presumably, international waters
# this is used below to determine whether any parts of pipelines are in international waters

if 'world_eez_and_land_boundaries_gdf' not in locals(): # only do this if it hasn't been done already
    world_eez_and_land_boundaries_gdf = shapely.ops.cascaded_union(eez_and_land_boundaries_gdf['geometry'])

#check type, should be multipolygon object
type(world_eez_and_land_boundaries_gdf)

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_3042/1801181543.py:6: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  world_eez_and_land_boundaries_gdf = shapely.ops.cascaded_union(eez_and_land_boundaries_gdf['geometry'])


shapely.geometry.multipolygon.MultiPolygon

# Import and clean data

In [6]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

# delete columns that aren't the same in the sheets, to concatenate them...
columns_not_in_oil = list(set(gas_pipes.columns)-set(oil_pipes.columns))
columns_not_in_gas = list(set(oil_pipes.columns)-set(gas_pipes.columns))
#gas_pipes.drop(columns=columns_not_in_oil, axis=1, inplace=True)
#oil_pipes.drop(columns=columns_not_in_gas, axis=1, inplace=True)

In [7]:
region_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df()

## replace eez_and_land_boundaries_gdf country names with the ones we use in GGIT/GOIT, for consistency

In [8]:
rename_eez_df = region_df_orig.copy()
rename_eez_df = rename_eez_df[rename_eez_df['EEZNamesIfDifferent']!='']
rename_eez_dict = dict(zip(rename_eez_df.EEZNamesIfDifferent, rename_eez_df.Country))
eez_and_land_boundaries_gdf.rename(index=rename_eez_dict, inplace=True)

## Specify Oil/NGL or Gas, concatenate if oil_and_gas

In [9]:
#type = 'Oil'
#type = 'Gas'
type = 'Oil_and_Gas'

if type=='Oil':
    pipe = oil_pipes
    #pipe.drop(column='CapacityBOEd', inplace=True)
elif type=='Gas':
    pipe = gas_pipes
    #pipe.drop('CapacityBcm/y', inplace=True)
elif type=='Oil_and_Gas':  
    pipe_orig_df = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

pipe_orig_df.replace('--', numpy.nan, inplace=True)

In [10]:
# get pipeline list, import as df
pipe_orig_df['PipelineName'] = pipe_orig_df['PipelineName'].str.strip()
pipe_orig_df['SegmentName'] = pipe_orig_df['SegmentName'].str.strip()
pipe_orig_df['ProjectID'] = pipe_orig_df['ProjectID'].str.strip()

# clean up column 'Route'
pipe_orig_df['Route'] = pipe_orig_df['Route'].str.strip()

# get rid of "N/A" and any empty routes (which would be empty rows)
pipe_orig_df = pipe_orig_df.loc[pipe_orig_df['Status']!='N/A']
pipe_orig_df = pipe_orig_df.loc[pipe_orig_df['Wiki']!='']
#pipe_orig = pipe_orig[pipe_orig['Route']!='']

missing_route_options = ['Unavailable',
                         'Capacity expansion only',
                         'Bidirectionality upgrade only',
                         'Short route (< 100 km)',
                         '']

pipes_noroute_df = pipe_orig_df.loc[pipe_orig_df['Route'].isin(missing_route_options)]
pipes_noroute_df = pipes_noroute_df.loc[pipes_noroute_df['Wiki']!=''] # get rid of anything without a wiki page
pipes_withroute_df = pipe_orig_df.loc[~pipe_orig_df['Route'].isin(missing_route_options)]

# Length Calculation Functions

## convert gfit to linestring

In [11]:
def convert_gfit_to_linestring(coord_str, pipeline_name, segment_name, project_id, status, fuel, length):
    '''
    Takes string from GFIT column of coordinates for a single pipeline,
    converts that string into Shapely LineString or MultiLinestring for processing.
    '''

    #print(pipeline_name, segment_name)
    #print(coord_str)
    if ':' in coord_str and ';' not in coord_str:
        # simple geometry; no branching
        # create nested list of lists, separating on colons        
        coord_list = coord_str.split(':')
        
        coord_list_tuples = []
        
        # non-branched pipeline (nested list with one level)
        # convert nested list of lists to list of tuples
        for element in coord_list:
            element_tuple = (float(element.split(',')[1]), 
                             float(element.split(',')[0]))
            coord_list_tuples.append(element_tuple)
            
        pipeline = shapely.geometry.LineString(coord_list_tuples)

    elif ':' in coord_str and ';' in coord_str:
        # create a nested list of lists, separating on semicolons
        coord_list = coord_str.split(';')
        
        # create a second level of nesting, separating on colons
        coord_list = [x.split(':') for x in coord_list]
        
        # branched pipeline (nested list with two levels)
        pipeline_ls_all = []
        
        for nested_list in coord_list:
            coord_list_tuples = []
            
            for element in nested_list:
                element_tuple = (float(element.split(',')[1]), 
                                 float(element.split(',')[0]))
                coord_list_tuples.append(element_tuple)
                
            # process coord_list_tuples
            try:
                pipeline_ls = shapely.geometry.LineString(coord_list_tuples)
                pipeline_ls_all.append(pipeline_ls)
            except:
                print(f"Exception for coord_list_tuples: {coord_list_tuples}") # for db
                pass
            
        pipeline = shapely.geometry.MultiLineString(pipeline_ls_all)
        
    else:
        # create empty MultiLineString; coordinates were missing or misformatted
        pipeline = shapely.geometry.MultiLineString([])
        
        print(f'Missing or misformatted coordinates for {pipeline_name} - {segment_name}')
        
    return pipeline

## pipeline total length and wiggle

In [12]:
def pipeline_total_length_and_wiggle(pipes_df):
    '''
    Iterate through each pipeline, calculating the total length and wiggle factor.
    
    Modifies the main df that was function argument, returning modified version.
    '''
    
    mask_route_1 = pipes_df['Route'].str.contains(',')
    mask_route_2 = pipes_df['Route'].str.contains(':')
    pipes_with_route = pipes_df.loc[(mask_route_1) & (mask_route_2)]
    
    for row in pipes_with_route.index:
        # get string with coordinates for route, convert to LineString (or MultiLineString)
        pipeline_name = pipes_with_route.at[row, 'PipelineName']
        segment_name = pipes_with_route.at[row, 'SegmentName']
        project_id = pipes_with_route.at[row, 'ProjectID']
        pipeline_str = pipes_with_route.at[row, 'Route']
        status = pipes_with_route.at[row, 'Status']
        fuel = pipes_with_route.at[row, 'Fuel']
        length = pipes_with_route.at[row, 'LengthMergedKm']
        pipeline_ls = convert_gfit_to_linestring(pipeline_str, pipeline_name, segment_name, 
                                                 project_id, status, fuel, length)

        # calculate length of LineString (or MultiLineString)
        geodetic_computation = pyproj.Geod(ellps="WGS84")
        length_calc = geodetic_computation.geometry_length(pipeline_ls)/1000 # units km

        # get reported length of pipeline
        length_report = pipes_with_route.at[row, 'LengthKnownKm']

        #print(pipeline_name, segment_name)
        
    #    if pandas.notnull(length_report):
    #        # calculate wiggle factor regardless of relationship,
    #        # whether length_report is > or < length_calc
    #        # if reported and calculated length both exist, calculate their ratio:
        try:
            wiggle_factor = length_report / length_calc
            pipes_df.at[row, 'WiggleFactor'] = wiggle_factor
        # if one doesn't exist, you get a TypeError when dividing; replace with
        except TypeError:
            print(pipeline_name, segment_name, project_id)
            print(length_report, length_calc)
            print('TypeError, WiggleFactor set to 1.0')
            pipes_df.at[row, 'WiggleFactor'] = float(1)

    #    else:
    #        print('notnull)')
    #        # there was no reported length; assign wiggle_factor = 1.0
    #        pipes_df.at[row, 'WiggleFactor'] = float(1)

    return(pipes_df)

## pipeline within country

In [13]:
eez_and_land_boundaries_gdf[eez_and_land_boundaries_gdf.index=='Hong Kong']

,OBJECTID,ISO_3digit,Changes,Shape_Leng,Shape_Area,geometry
Hong Kong,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((114.22983 22.55581, 114.23471 ..."


In [14]:
def pipeline_within_country(pipeline_ls, 
                            pipeline_name, 
                            segment_name, 
                            project_id, 
                            results_by_country, 
                            status, 
                            fuel, 
                            length, 
                            remainders, 
                            international):
    '''
    Iterate through all countries, to see if the specified pipeline 
    is within each country (at least partially).
    
    If there is a portion within a given country, 
    saves the country name and length of pipeline to a df
    '''
    
    pipeline_remainders = pipeline_ls # initialize
    # will progressively remove pieces of the pipeline, 
    # as they intersect with each country's land mass
    
    geodetic_computation = pyproj.Geod(ellps="WGS84") # initialize
    
    length_total = geodetic_computation.geometry_length(pipeline_ls)/1000 # units km
    
    for country in eez_and_land_boundaries_gdf.index:
        
        country_geom = eez_and_land_boundaries_gdf.loc[country, 'geometry']
        
        if country_geom.intersects(pipeline_ls)==True:
            pipeline_intersection = pipeline_ls.intersection(country_geom)
            pipeline_remainders = pipeline_remainders.difference(country_geom)
            
            length_per_country = geodetic_computation.geometry_length(pipeline_intersection)/1000 # units km
            length_per_country_fract = length_per_country / length_total
            
            one_result = (
                pipeline_name, 
                segment_name,
                project_id,
                country, 
                length_per_country, 
                length_per_country_fract,
                status,
                fuel,
                length)
            one_result_df = pandas.DataFrame(one_result).T
            one_result_df.columns = ['pipeline_name', 'segment_name', 'project_id', 'country', 
                                     'length_per_country', 'length_per_country_fract', 
                                     'status', 'fuel', 'length']
            
            results_by_country = results_by_country.append(
                pandas.DataFrame(one_result_df), 
                sort=False)
            
        else:
            pass

    results_by_country = results_by_country.reset_index(drop=True)
        
    if pipeline_remainders.is_empty==False:       
        remainders_length = geodetic_computation.geometry_length(pipeline_remainders)/1000 # units km
        
        if remainders_length > 0.01: # units: km
            remainders_tuple = (pipeline_name, remainders_length, pipeline_remainders)
            remainders = pandas.DataFrame(remainders_tuple).T
            remainders.columns = ['pipeline_name', 'segment_name', 'project_id', 'length', 'geometry']
        
            print(f"for {pipeline_name}, pipeline_remainders.is_empty==False") # for db
            print(f"remainders_length: {remainders_length}") # for db
        
    else:
#         print(f"for {pipeline_name}, pipeline_remainders.is_empty is NOT False") # for db
        pass
    
    # alternative method: pipeline that's in international waters (not in world_eez_and_land_boundaries_gdf)
    international_pipeline = pipeline_ls.difference(world_eez_and_land_boundaries_gdf)
    if international_pipeline.is_empty==False:
        international_length = geodetic_computation.geometry_length(international_pipeline)/1000 # units km
        international_tuple = (pipeline_name, international_length, international_pipeline)
        international = pandas.DataFrame(international_tuple).T
        international.columns = ['pipeline_name', 'length', 'geometry']
    else:
        pass
    
    return results_by_country, remainders, international

# Apply functions to data

In [15]:
# this step requires that there be no non-geometry data in the "Route" column, 
# meaning no letters or extraneous symbols (ex: ";;", "::", "--", etc.)
pipe_gpd = geopandas.GeoDataFrame(pipes_withroute_df)
pipe_gpd['geometry'] = ''
for row in pipe_gpd.index:
    linestring = convert_gfit_to_linestring(
        str(pipe_gpd.at[row, 'Route']), 
        pipe_gpd.at[row, 'PipelineName'], 
        pipe_gpd.at[row, 'SegmentName'], 
        pipe_gpd.at[row, 'ProjectID'], 
        pipe_gpd.at[row, 'Status'], 
        pipe_gpd.at[row, 'Fuel'], 
        pipe_gpd.at[row, 'LengthMergedKm'])
    pipe_gpd.at[row, 'geometry'] = linestring

/Users/baird/miniconda3/envs/gem/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Exception for coord_list_tuples: [(-0.9540990052655393, 38.08500504384781)]


In [16]:
# calculate length by country
# get coord_str for each pipeline that has route coordinates
# choose pipes_withroute_df to process
#pipeline_df = pipe.copy()

results_by_country = pandas.DataFrame(
    columns=['pipeline_name', 'segment_name', 'project_id', 'country', 'length_per_country', 'status', 'fuel', 'length']
)

remainders = geopandas.GeoDataFrame()
international = geopandas.GeoDataFrame()

for sel_index in pipes_withroute_df.index:
    pipeline_name = pipes_withroute_df.at[sel_index, 'PipelineName']
    segment_name = pipes_withroute_df.at[sel_index, 'SegmentName']
    project_id = pipes_withroute_df.at[sel_index, 'ProjectID']
    
    #print(pipeline_name, segment_name, project_id)
    
    status = pipes_withroute_df.at[sel_index, 'Status']
    fuel = pipes_withroute_df.at[sel_index, 'Fuel']
    length = pipes_withroute_df.at[sel_index, 'LengthKnownKm']
    coord_str = str(pipes_withroute_df.at[sel_index, 'Route'])
    
    pipeline_ls = convert_gfit_to_linestring(coord_str, 
                                             pipeline_name, 
                                             segment_name, 
                                             project_id, 
                                             status, 
                                             fuel, 
                                             length)
    
    results_by_country, remainders, international = pipeline_within_country(
        pipeline_ls, 
        pipeline_name, 
        segment_name, 
        project_id, 
        results_by_country, 
        status, 
        fuel, 
        length, 
        remainders, 
        international)

Exception for coord_list_tuples: [(-0.9540990052655393, 38.08500504384781)]


# Now go through all pipelines that DON'T have a route and fill in missing length info

In [17]:
results_by_country_noroute = pandas.DataFrame(
    columns=['pipeline_name', 'segment_name', 'project_id', 'country', 'length_per_country', 'status', 'fuel', 'length']
)

for sel_index in pipes_noroute_df.index:

    pipeline_name = pipes_noroute_df.at[sel_index, 'PipelineName']
    segment_name = pipes_noroute_df.at[sel_index, 'SegmentName']
    project_id = pipes_noroute_df.at[sel_index, 'ProjectID']
    status = pipes_noroute_df.at[sel_index, 'Status']
    fuel = pipes_noroute_df.at[sel_index, 'Fuel']
    length = pipes_noroute_df.at[sel_index, 'LengthKnownKm']
    coord_str = str(pipes_noroute_df.at[sel_index, 'Route'])
    
    #print(pipeline_name, segment_name, coord_str, length)
    
    ncountries = pipes_noroute_df.at[sel_index, 'NumberOfCountries']

    # if it crosses into more than one country
    if ncountries>1:

        country_list = pipes_noroute_df.at[sel_index, 'Countries'].split(',')
        country_list = [i.strip() for i in country_list]

        for country in country_list:
            length_per_country = length/country_list.__len__()
            length_per_country_fract = 1/country_list.__len__()

            one_result = (
                pipeline_name, 
                segment_name,
                project_id,
                country, 
                length_per_country, 
                length_per_country_fract,
                status,
                fuel,
                length)
            one_result_df = pandas.DataFrame(one_result).T
            one_result_df.columns = ['pipeline_name', 'segment_name', 'project_id', 'country', 
                                     'length_per_country', 'length_per_country_fract', 
                                     'status', 'fuel', 'length']

            results_by_country_noroute = pandas.concat([results_by_country_noroute,
            pandas.DataFrame(one_result_df)], axis=0,
            sort=False)

    else:
        country = pipes_noroute_df.at[sel_index, 'Countries']

        length_per_country = length
        length_per_country_fract = 1.0

        one_result = (
            pipeline_name, 
            segment_name,
            project_id,
            country, 
            length_per_country, 
            length_per_country_fract,
            status,
            fuel,
            length)
        one_result_df = pandas.DataFrame(one_result).T
        one_result_df.columns = ['pipeline_name', 'segment_name', 'project_id', 'country', 
                                 'length_per_country', 'length_per_country_fract', 
                                 'status', 'fuel', 'length']

        results_by_country_noroute = pandas.concat([results_by_country_noroute,
            pandas.DataFrame(one_result_df)], axis=0,
            sort=False)

In [31]:
results_by_country_combined = pandas.concat([results_by_country,
                                             results_by_country_noroute],
                                            axis=0, sort=False)

# Check Mean Wiggle Factor, & Outliers

# Clean and export results

In [32]:
# export length estimates by country and pipeline
results_by_country_combined = results_by_country_combined[['pipeline_name','segment_name',
                                                          'project_id', 'country', 'length_per_country', 
                                                           'length', 'length_per_country_fract']]
results_by_country_combined.rename(columns={
                                    'pipeline_name':'PipelineName',
                                    'segment_name':'SegmentName',
                                    'project_id':'ProjectID',
                                    'country':'Country',
                                    'length_per_country':'LengthEstimateKmByCountry',
                                    'length':'LengthEstimateKm',
                                    'length_per_country_fract':'LengthPerCountryFraction'}, inplace=True)

results_by_country_combined.sort_values('ProjectID', inplace=True)
# results_by_country['Country'].replace("Cote d'Ivoire", "Côte d'Ivoire", inplace=True)
# results_by_country['Country'].replace('Czech Republic', 'Czechia', inplace=True)
# results_by_country['Country'].replace('Swaziland', 'Eswatini', inplace=True)
# results_by_country['Country'].replace('Congo', 'Republic of Congo', inplace=True)
# results_by_country['Country'].replace('Congo, DRC', 'DR Congo', inplace=True)
# results_by_country['Country'].replace('Sudan', 'Republic of Sudan', inplace=True)
# results_by_country['Country'].replace('Guinea', 'Republic of Guinea', inplace=True)
# results_by_country['Country'].replace('Bosnia & Herzegovina', 'Bosnia and Herzegovina', inplace=True)
# results_by_country['Country'].replace('Trinidad & Tobago', 'Trinidad and Tobago', inplace=True)

results_by_country_combined.to_excel('Estimated_Length-Results_By_Country_'+type+'.xlsx')

In [33]:
results_by_country_combined

,PipelineName,SegmentName,ProjectID,Country,LengthEstimateKmByCountry,LengthEstimateKm,LengthPerCountryFraction
0,Alberta Clipper Oil Pipeline,,P0001,Canada,1099.524365,1790.0,0.703393
1,Alberta Clipper Oil Pipeline,,P0001,United States,463.648701,1790.0,0.296608
2,Athabasca Oil Pipeline,,P0002,Canada,418.095904,542.35,1.0
3,Bakken Expansion Pipeline,,P0004,Canada,155.333654,260.71,0.592423
4,Bakken Expansion Pipeline,,P0004,United States,106.866868,260.71,0.407577
...,...,...,...,...,...,...,...
3900,Ilichevsk-Yerevan Gas Pipeline,,P4271,Armenia,59.092131,86.6,0.729526
3901,Ilichevsk-Yerevan Gas Pipeline,,P4271,Azerbaijan,21.908711,86.6,0.270476
3902,Yufutsu-Sapporo Gas Pipeline,,P4272,Japan,43.007411,72.0,1.0
0,Shin Oak NGL Pipeline,Capacity Expansion,P4277,United States,NaN,NaN,1.0


## create the estmated length by each pipeline (has "--" where no route exists)

In [34]:
results_by_pipeline_withroute = results_by_country.copy()[['project_id','length_per_country']]
results_by_pipeline_withroute = pandas.DataFrame(results_by_country.groupby("project_id")["length_per_country"].sum())

# then add in the missing route project_ids and set the length to '--'
noroute_projectids = list(set(results_by_country_noroute['project_id']))

results_by_pipeline_noroute = pandas.DataFrame({'length_per_country':['--']*noroute_projectids.__len__()},
                                              index=noroute_projectids)
results_by_pipeline_noroute.index.name = 'project_id'

results_by_pipeline_combined = pandas.concat([results_by_pipeline_withroute,results_by_pipeline_noroute])
results_by_pipeline_combined.sort_index(inplace=True)
results_by_pipeline_combined.index.name = 'ProjectID'
results_by_pipeline_combined.rename({'length_per_country':'LengthEstimateKmByCountry'})

#results_by_pipeline_combined = results_by_country_noroute.copy()[['project_id','length_per_country']]
results_by_pipeline_combined.to_excel('Estimated_Length-Results_By_Pipeline_'+type+'.xlsx')